# Parsimonious flux balance analysis (pFBA)

Parsimonious flux balance analysis attempts to find a realistic flux of a
model, by trying to minimize squared sum of all fluxes while maintaining the
reached optimum. COBREXA.jl implements it in function
`parsimonious_flux_balance_analysis` (accompanied by vector- and
dictionary-returning variants
`parsimonious_flux_balance_analysis_vec` and
`parsimonious_flux_balance_analysis_dict`).

As usual, we demonstrate the functionality on the *E. coli* model:

In [1]:
!isfile("e_coli_core.xml") &&
    download("http://bigg.ucsd.edu/static/models/e_coli_core.xml", "e_coli_core.xml")

using COBREXA, Tulip, OSQP

model = load_model("e_coli_core.xml")

Metabolic model of type SBMLModel
sparse([41, 23, 51, 67, 61, 65, 1, 7, 19, 28  …  72, 3, 8, 33, 57, 66, 31, 45, 46, 57], [1, 2, 2, 2, 3, 3, 4, 4, 4, 4  …  93, 94, 94, 94, 94, 94, 95, 95, 95, 95], [-1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0  …  1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0], 72, 95)
Number of reactions: 95
Number of metabolites: 72


Because the parsimonious objective is quadratic, we need a an optimizer
capable of solving quadratic programs.

As the simplest choice, we can use
[`OSQP.jl`](https://osqp.org/docs/get_started/julia.html), but any any
[`JuMP.jl`-supported
optimizer](https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers)
that supports quadratic programming will work.

Running of basic pFBA is perfectly analogous to running of [FBA](05a_fba.md)
and other analyses. We add several modifications that improve the solution
(using functions `silence`, and
`change_optimizer_attribute`), and fix the glucose exchange (using
`change_constraint`) in order to get a more reasonable result:

In [2]:
fluxes = parsimonious_flux_balance_analysis_dict(
    model,
    OSQP.Optimizer;
    modifications = [
        silence, # optionally silence the optimizer (OSQP is very verbose by default)
        change_optimizer_attribute("polish", true), # tell OSQP to invest time into improving the precision of the solution
        change_constraint("R_EX_glc__D_e"; lb = -12, ub = -12), # fix glucose consumption rate
    ],
)

Dict{String, Float64} with 95 entries:
  "R_EX_fum_e"    => -0.0054306
  "R_ACONTb"      => 6.51108
  "R_TPI"         => 8.90211
  "R_SUCOAS"      => -5.41568
  "R_GLNS"        => 0.250914
  "R_EX_pi_e"     => -3.85013
  "R_PPC"         => 2.94799
  "R_O2t"         => 25.1823
  "R_G6PDH2r"     => 6.27109
  "R_TALA"        => 1.90314
  "R_PPCK"        => -0.00186526
  "R_EX_lac__D_e" => -0.00399055
  "R_PGL"         => 6.27111
  "R_H2Ot"        => -33.9472
  "R_GLNabc"      => 0.0126618
  "R_EX_co2_e"    => 26.4219
  "R_EX_gln__L_e" => -0.0126492
  "R_EX_nh4_e"    => -5.67116
  "R_MALt2_2"     => 0.00493596
  ⋮               => ⋮

## Using different optimizers for linear and quadratic problems

It is quite useful to use specialized optimizers for specialized tasks in
pFBA. In particular, one would usually require to get a precise solution from
the linear programming part (where the precision is achievable), and trade
off a little precision for vast improvements in computation time in the
quadratic programming part.

In pFBA, we can use the `modifications` and `qp_modifications` parameters to
switch and parametrize the solvers in the middle of the process, which allows
us to implement precisely that improvement. We demonstrate the switching on a
vector-returning variant of pFBA:

In [3]:
flux_vector = parsimonious_flux_balance_analysis_vec(
    model,
    Tulip.Optimizer; # start with Tulip
    modifications = [
        change_constraint("R_EX_glc__D_e"; lb = -12, ub = -12),
        change_optimizer_attribute("IPM_IterationsLimit", 500), # we may change Tulip-specific attributes here
    ],
    qp_modifications = [
        change_optimizer(OSQP.Optimizer), # now switch to OSQP (Tulip wouldn't be able to finish the computation)
        change_optimizer_attribute("polish", true), # get an accurate solution, see OSQP's documentation
        silence, # and make it quiet.
    ],
)

95-element Vector{Float64}:
  -0.006231402533094581
   6.847122967587614
   8.914435875871503
  -5.738183656745528
   0.25363156710733
  -3.8887743417980833
   2.977310372502161
  25.64208341306474
   6.1991009754149715
   1.877264043236841
   ⋮
  -0.0002197051126034427
  34.49592944330335
  -0.0020752886988600542
  -0.0014018715428246171
  -0.002009900868927077
 -25.642083259145206
   0.0161569956400718
   0.004729690379188246
   3.8887743479784547

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*